In [2]:
import pymysql

In [3]:
def connect():
    con = pymysql.connect(
        host = 'localhost',
        port = 3306,
        user = 'root',
        password = 'gkdks120',
        db = 'ML_DB',
        charset = 'utf8'
    )
    print('1. 연결성공')
    cur = con.cursor()
    print('2. 커서 객체 획득')
    return cur,con

In [38]:
cur2, con = connect()
cur2

1. 연결성공
2. 커서 객체 획득


In [5]:
def sql_execute(cur2, data):
    sql = '''
    select o2.orderid, m.name, o2.productid, o2.totalprice
    from orderlist o2
    inner join `member` m
    on o2.memberid = m.id and m.id = %s
    order by productid desc
    '''
    result = cur2.execute(sql,data)
    return result

In [6]:
sql_result = sql_execute(cur2, 'park')
sql_result

4

In [7]:
## 1. 누가 어떤 물건을 얼마나 샀는가?

In [8]:
who = input('누가?? >> ')

누가?? >> park


In [9]:
sql_result2 = sql_execute(cur2,who)
sql_result2

4

In [10]:
def out_data(cur2):
    rows = cur2.fetchall()
    return rows

In [11]:
result_rows = out_data(cur2)
result_rows

((2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000))

In [12]:
def db_close(con):
    con.close()

In [13]:
db_close(con)

In [14]:
## 제일 높은 금액은?

In [17]:
[x[3] for x in result_rows]

[6000, 5000, 10000, 3000]

In [18]:
max([x[3] for x in result_rows])

10000

In [19]:
## 제일 많이 산 물건의 id는?

In [20]:
import pandas as pd

In [22]:
result_df = pd.DataFrame(data=result_rows,
                        columns=['주문id','name','제품id','주문총가격'])

In [23]:
result_df

,주문id,name,제품id,주문총가격
0,2,park,101,6000
1,1,park,100,5000
2,3,park,100,10000
3,4,park,100,3000


In [24]:
[x for x in result_df['제품id']]

['101', '100', '100', '100']

In [25]:
from collections import Counter

In [26]:
counter_id = Counter(result_df['제품id'])

In [27]:
counter_id

Counter({'101': 1, '100': 3})

In [28]:
max_id = counter_id.most_common(n=1)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('100', 3)]
100


'100'

In [29]:
def sql_execute2(cur2,data):
    sql = '''
    select * from product
    where id = %s
    '''
    result2 = cur2.execute(sql,data)
    return result2

In [30]:
cur2, con = connect()
cur2

1. 연결성공
2. 커서 객체 획득


In [31]:
sql_result2 = sql_execute2(cur2, max_id_result)
sql_result2

1

In [32]:
result_rows = out_data(cur2)
result_rows[0]

('100', 'java', 'fun java', '1000')

In [33]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('------------------------------------')
print('물건의 이름은 ',result_rows[0][1])
print('물건의 내용은', result_rows[0][2])
print('물건의 가격은', result_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
------------------------------------
물건의 이름은  java
물건의 내용은 fun java
물건의 가격은 1000


In [34]:
def sql_execute3(con,cur2,data):
    sql = '''
    insert into orderlist values (null,%s,%s,%s)
    '''
    result2 = cur2.execute(sql,data)
    con.commit()
    return result2

In [35]:
order_question = input('구매하시겠습니까?(yes/no)')
if order_question == 'yes':
    print('---------구매를 진행합니다.----------')
    your_id = input('당신의 id는 ')
    data_list = [your_id, result_rows[0][0], int(result_rows[0][3])]
    result3 = sql_execute3(con,cur2,data_list)
    if result3 == 1:
        print('---구매가 완료되었습니다.')
    else:
        print('---구매에 문제가 발생했습니다.')

구매하시겠습니까?(yes/no)yes
---------구매를 진행합니다.----------
당신의 id는 park
---구매가 완료되었습니다.


In [36]:
con.close()

In [ ]:
# 추천을 하고자 하는 컨텐츠기반 룰을 정하고,
# 물건을 3개 이상 추천하는 추천시스템을 완성하시오
# db의 데이터를 임의로 추가할 수 있음